In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.Monad

# Defining Custom Monads
We can define a `Monad` for a custom type by providing implementations of three methods:
- `flatMap`
- `pure`
- `tailRecM`

Here's an example of the implementation for `Option`:

In [ ]:
implicit val optionMonad: Monad[Option] = new Monad[Option] {
  def flatMap[A, B](opt: Option[A])(fn: A => Option[B]): Option[B] =
    opt.flatMap(fn)
  
  def pure[A](opt: A): Option[A] =
    Some(opt)

  @scala.annotation.tailrec
  def tailRecM[A, B](a: A)(fn: A => Option[Either[A, B]]): Option[B] =
    fn(a) match {
      case None => None
      case Some(Left(a1)) => tailRecM(a1)(fn)
      case Some(Right(b)) => Some(b)
    }
}

The `tailRecM` method is an optimization used in Cats. If we can make `tailRecM` tail-recursive, Cats can guarentee we won't get `StackOverflowErrors` in recursive situations like folding over a large list. All of the built-in monads in Cats have tail-recursive implementations of `tailRecM`. Writing a tail-recursive `tailRecM` for you custom monad is almost always possible, though sometimes it can be a challenge.

## Exercise

Let's implement `Monad` for `Tree`:

In [ ]:
sealed trait Tree[+A]

final case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

final case class Leaf[A](value: A) extends Tree[A]

def branch[A](left: Tree[A], right: Tree[A]): Tree[A] =
  Branch(left, right)

def leaf[A](value: A): Tree[A] =
  Leaf(value)

We'll need to verify the code works on instances of `Branch` and `Leaf`, and that the `Monad` provides `Functor`-like behavior for free. Implementing `tailRecM` here is very difficult, so don't feel bad if you just want to examine the solution.

In [ ]:
implicit val treeMonad: Monad[Tree] = new Monad[Tree] {
  def pure[A](value: A): Tree[A] =
    ???

  def flatMap[A, B](tree: Tree[A])(func: A => Tree[B]): Tree[B] =
    ???

  // @scala.annotation.tailrec
  def tailRecM[A, B](arg: A)(func: A => Tree[Either[A, B]]): Tree[B] =
    ???
}

// Lets see if we get map for free
val mapDemo = branch(leaf(100), leaf(200)).map(_ + 2)

// Lets see if we can use it in a for comprehension
val forDemo = for {
  a <- branch(leaf(100), leaf(200))
  b <- branch(leaf(a - 10), leaf(a + 10))
  c <- branch(leaf(b - 1), leaf(b + 1))
} yield c

In [ ]:
implicit val treeMonad: Monad[Tree] = new Monad[Tree] {
  def pure[A](value: A): Tree[A] =
    Leaf(value)

  def flatMap[A, B](tree: Tree[A])(func: A => Tree[B]): Tree[B] =
    tree match {
      case Branch(l, r) =>
        Branch(flatMap(l)(func), flatMap(r)(func))
      case Leaf(value) =>
        func(value)
    }

  def tailRecM[A, B](arg: A)(func: A => Tree[Either[A, B]]): Tree[B] = {
    
    @scala.annotation.tailrec
    def loop(open: List[Tree[Either[A, B]]], closed: List[Option[Tree[B]]]): List[Tree[B]] =
      open match {
        case Branch(l, r) :: next =>
          loop(l :: r :: next, None :: closed)
        
        case Leaf(Left(value)) :: next =>
          loop(func(value) :: next, closed)
        
        case Leaf(Right(value)) :: next =>
          loop(next, Some(pure(value)) :: closed)
        
        case Nil =>
          closed.foldLeft(Nil: List[Tree[B]]) { (acc, maybeTree) =>
            maybeTree.map(_ :: acc).getOrElse {
              val left :: right :: tail = acc
              branch(left, right) :: tail
            }
          }
      }
    
    loop(List(func(arg)), Nil).head
  }
}

// Lets see if we get map for free
val mapDemo = branch(leaf(100), leaf(200)).map(_ + 2)

// Lets see if we can use it in a for comprehension
val forDemo = for {
  a <- branch(leaf(100), leaf(200))
  b <- branch(leaf(a - 10), leaf(a + 10))
  c <- branch(leaf(b - 1), leaf(b + 1))
} yield c

What are the semantics of `flatMap` for `Tree`? Phrased differently, what *complication* does `Tree` implement?

Every node in the tree has the potential to be replaced with a whole subtree, producing a kind of "growing" or "feathering" behavior.

## Other Monads in Cats
`IO`

An *effect* monad. Better replacement for `Future`. Makes all code referentially transparent. It's *complication* is side-effecting.

`NonEmptyList`

Same as list, but represents a non-empty set of intermediate results.

`Chain`

Alternative to `List` and `Vector` with different performance characteristics.

`Ior`

Inclusive-or. Similar to Either, but also has the ability to be *both*. Can be thought of as `Result`, `Error`, or `ResultAndWarning`. 

`Kleisli`

Same thing as `ReaderT` which is basically an alias for `A => F[B]`. Very handy and not nearly as intimidating as its name.

`Free`

Turn any sealed hierarchy into a monad. Get the result by converting to a real monad at a later time with an *interpreter*.

`And more...`

Many more from popular libraries such as `Stream` from fs2, `ConnectionIO` from doobie, and `Parser` from atto.